In [1]:
import pandas as pd
import win32com.client
from datetime import datetime
import numpy as np

의문점
 Cybos를 자동으로 실행하는 방법.
 복수 주문을 할 때 setting을 다 박아 놓고 request를 하면 되는 것인가?

In [2]:
def connection():
    obj = win32com.client.Dispatch('CpUtil.CpCybos')
    if obj.Isconnect== 1:
        print('Cybos가 정상적으로 연결되었습니다')
    else:
        print('연결 안 됨')        
        
def trade(order):
    for idx, order_stock in order.iterrows():
        codeName, short_code,  = order_stock['codeName'], order_stock['short_code']        
        order_quantity, order_price = order_stock['quantity'], order_stock['price']
        if order_quantity > 0:
            buysell = '매수'
        else:
            buysell = '매도'
        print('{} 종목 {}주 {}원에 {}'.format(codeName, order_quantity, order_price, buysell))
        stockorder = trd_stock_setting(short_code, order_quantity, order_price)
        request(stockorder)


def trd_stock_setting(short_code, order_quantity, order_price):
    tradeobj = win32com.client.Dispatch('CpTrade.CpTdUtil')
    initCheck = tradeobj.TradeInit(0) # 주문 초기화 / 
    acc = tradeobj.AccountNumber[0] # 계좌 번호
    goods_type = tradeobj.GoodsList(acc,1)[0] # 1은 주식, 2는 선물/옵션
    
    stockorder = win32com.client.Dispatch('CpTrade.CpTd0311')    
    if order_quantity > 0:  # 매도 1, 매수 2
        buysell = '2'
    else:
        buysell = '1'     
    acc_num = acc # 계좌번호
    goods_type = '10' # 주식 상품 구분
    short_code = short_code # SK 하이닉스 종목코드
    quantity = abs(order_quantity) # 수량
    price = order_price # 호가
    order_type = '0' # 0은 기본값, IOC는 1, FOK는 2
    call_type = '01' # 지정가 01, 임의 02, 시장가 03, 조건부지정가 05, 최유리 12, 최우선 13
    
    
    stockorder.SetInputValue(0,buysell) 
    stockorder.SetInputValue(1,acc_num)
    stockorder.SetInputValue(2,goods_type)
    stockorder.SetInputValue(3,short_code)
    stockorder.SetInputValue(4,quantity)
    stockorder.SetInputValue(5,price)
    # stockorder.SetInputValue(6,)
    stockorder.SetInputValue(7, order_type)
    stockorder.SetInputValue(8,call_type)
    
    return stockorder

def request(stockorder):
    stockorder.BlockRequest()
    st_msg = stockorder.GetDibMsg1()
    print(st_msg)

    
def check_contract():
    account = win32com.client.Dispatch('CpTrade.CpTd5341')

    sort_type = '0' # 순차 0, 역순 1
    num_data = 20 # default 7, maximum 20
    inq_type = '1' # 단가별 : 1, 건별 : 2, 합산 : 3

    tradeobj = win32com.client.Dispatch('CpTrade.CpTdUtil')
    acc = tradeobj.AccountNumber[0] # 계좌 번호
    goods_type = '10'
    num_data = 20
    inq_type = '1'

    account.SetInputValue(0,acc) # 계좌번호
    account.SetInputValue(1, goods_type) #상품관리구분코드 
    #account.SetInputValue(2,) # 종목코드 / 생략 시 전 종목
    #account.SetInputValue(3,) # 시작주문번호 / 생략 시 전 종목
    account.SetInputValue(4,sort_type) # 정렬 / 0은 순차, 1은 역순
    account.SetInputValue(5,num_data) # 요청개수 / 기본 7개 최대 20개
    account.SetInputValue(6, inq_type) # 조회구분 / 1 단가별 2 건별, 3 합산

    account.BlockRequest()

    orders_list = []
    col = ['상품구분','주문번호', '원주문번호', '종목코드', '종목이름', '주문내용', 
           '주문수량', '주문단가', '총체결수량', '체결수량', '체결단가', '확인수량']
    for i in range(len(order)):
        order_list = []
        for j in range(13):
            if j==6:
                continue
            order_list.append(account.GetDataValue(j,i))
        orders_list.append(order_list)

    account_df = pd.DataFrame(orders_list, columns = col)
    
    def to_str_value(int_type):
        str_type = str(int_type)
        split_list = []
        for i in range(0,len(str_type),3):
            if i==0:
                split_list.append(str_type[-i-3:])
            else:
                split_list.append(str_type[-i-3:-i])
        value_str = ','.join(split_list[::-1]) + '원'
        return value_str

    portfolio_value = np.array(account_df['총체결수량']).dot(np.array(account_df['체결단가']))
    portfolio_value_str = to_str_value(portfolio_value)
    return account_df   

In [3]:
connection()

Cybos가 정상적으로 연결되었습니다


In [ ]:
if __name__ == '__main__':
    today = datetime.today().strftime('%Y%m%d')
    order = pd.read_pickle('order20200128.pickle')
    new_col = ['checker', 'short_code', 'codeName', 'slope','weight', 'allocation', 'price', 'quantity']
    order.columns = new_col
    order[['allocation', 'price']] = order[['allocation', 'price']].applymap(lambda x : int(x))
    
    connection()
    trade(order) 

In [35]:
order.head()

,checker,short_code,codeName,slope,weight,allocation,price,quantity
0,True,A171010,램테크놀러지,-0.044366,0.063711,19113373,7710,2479
1,True,A102940,코오롱생명과학,-0.041031,0.058922,17676703,18500,955
2,True,A005950,이수화학,-0.036042,0.051758,15527519,9720,1597
3,True,A042420,네오위즈홀딩스,-0.035777,0.051377,15413151,14600,1055
4,True,A009730,코센,-0.035741,0.051325,15397489,1210,12725


In [126]:
account = win32com.client.Dispatch('CpTrade.CpTd5341')

sort_type = '0' # 순차 0, 역순 1
num_data = 20 # default 7, maximum 20
inq_type = '1' # 단가별 : 1, 건별 : 2, 합산 : 3

tradeobj = win32com.client.Dispatch('CpTrade.CpTdUtil')
acc = tradeobj.AccountNumber[0] # 계좌 번호
goods_type = '10'
num_data = 20
inq_type = '1'

account.SetInputValue(0,acc) # 계좌번호
account.SetInputValue(1, goods_type) #상품관리구분코드 
#account.SetInputValue(2,) # 종목코드 / 생략 시 전 종목
#account.SetInputValue(3,) # 시작주문번호 / 생략 시 전 종목
account.SetInputValue(4,sort_type) # 정렬 / 0은 순차, 1은 역순
account.SetInputValue(5,num_data) # 요청개수 / 기본 7개 최대 20개
account.SetInputValue(6, inq_type) # 조회구분 / 1 단가별 2 건별, 3 합산

account.BlockRequest()

orders_list = []
col = ['상품구분','주문번호', '원주문번호', '종목코드', '종목이름', '주문내용', '주문수량', '주문단가', '총체결수량', '체결수량', '체결단가', '확인수량']
for i in range(len(order)):
    order_list = []
    for j in range(13):
        if j==6:
            continue
        order_list.append(account.GetDataValue(j,i))
    orders_list.append(order_list)

account_df = pd.DataFrame(orders_list, columns = col)

def to_str_value(int_type):
    str_type = str(int_type)
    split_list = []
    for i in range(0,len(str_type),3):
        if i==0:
            split_list.append(str_type[-i-3:])
        else:
            split_list.append(str_type[-i-3:-i])
    value_str = ','.join(split_list[::-1]) + '원'
    return value_str

portfolio_value = np.array(account_df['총체결수량']).dot(np.array(account_df['체결단가']))
portfolio_value_str = to_str_value(portfolio_value)
print(portfolio_value_str, account_df)
    

,상품구분,주문번호,원주문번호,종목코드,종목이름,주문내용,주문수량,주문단가,총체결수량,체결수량,체결단가,확인수량
0,10,53,0,A171010,램테크놀러지,정규장현금매수,2479,7710,2479,2479,7372,0
1,10,54,0,A102940,코오롱생명과학,정규장현금매수,955,18500,955,955,17950,0
2,10,55,0,A005950,이수화학,정규장현금매수,1597,9720,1597,1597,9472,0
3,10,56,0,A042420,네오위즈홀딩스,정규장현금매수,1055,14600,1055,1055,14294,0
4,10,57,0,A009730,코센,정규장현금매수,12725,1210,12725,12725,1159,0
5,10,58,0,A114090,GKL,정규장현금매수,733,21000,733,733,19100,0
6,10,59,0,A143210,핸즈코퍼레이션,정규장현금매수,2037,7430,2037,2037,7401,0
7,10,60,0,A086040,바이오톡스텍,정규장현금매수,2281,6580,0,0,0,0
8,10,61,0,A011090,에넥스,정규장현금매수,12390,1200,0,0,0,0
9,10,62,0,A143160,아이디스,정규장현금매수,579,25450,579,579,23900,0


In [129]:
portfolio_value_str = to_str_value(portfolio_value)
portfolio_value_str

'246,344,880원'

In [115]:
portfolio_value_str = '23,008,612,221,415'

In [128]:
def to_str_value(int_type):
    str_type = str(int_type)
    split_list = []
    for i in range(0,len(str_type),3):
        if i==0:
            split_list.append(str_type[-i-3:])
        else:
            split_list.append(str_type[-i-3:-i])
    value_str = ','.join(split_list[::-1]) + '원'
    return value_str

In [40]:
connection()

Cybos가 정상적으로 연결되었습니다


In [41]:
account = win32com.client.Dispatch('CpTrade.CpTd6033')

In [42]:
acc

'335029644'

In [43]:
goods_type

'10'

In [44]:
account.SetInputValue(0, acc)
account.SetInputValue(1, goods_type)
account.SetInputValue(2, 50)
account.SetInputValue(3, '2')

In [45]:
account.BlockRequest()

0

In [77]:
account.GetDataValue(0,18)

'효성티앤씨'

In [122]:
unused_list = [1,2,13,14,16]
com_cnt = 0
while True:
    com_cnt += 1
    try:
        account.GetDataValue(0,com_cnt)
    except Exception as e:
        print('portfolio 내 종목개수 : {}개'.format(com_cnt))
        break

portfolio 내 종목개수 : 19개


In [94]:
com_cnt

19

In [102]:
df = []
for i in range(com_cnt):
    row = []
    for j in range(19):        
        if j in unused_list:
            continue
        row.append(account.GetDataValue(j,i))
    df.append(row)


In [118]:
col = ['종목명', '결제잔고', '결제장부단가', '전일체결수량', ' 금일체결수량', ' 체결잔고수량', ' 장부가', ' 평가금액',
 '평가손익', '수익률', '종목코드', ' 매도가능수량', ' 체결장부단가', ' 손익단가']

In [120]:
df_account = pd.DataFrame(df, columns = col)

In [121]:
df_account.head()

,종목명,결제잔고,결제장부단가,전일체결수량,금일체결수량,체결잔고수량,장부가,평가금액,평가손익,수익률,종목코드,매도가능수량,체결장부단가,손익단가
0,SK하이닉스,10,99300,0,0,10,99536,992139000,-3221000,-0.323670,A000660,10,99536.124829,100022
1,삼성전자,10,61600,0,0,10,61773,601775000,-15955000,-2.582871,A005930,10,61773.054274,62095
2,이수화학,1597,9472,0,0,1597,9486,14650205000,-499294000,-3.295781,A005950,1597,9486.223844,9522
3,한창제지,4616,3080,0,0,4616,3084,12528683000,-1709316000,-12.005311,A009460,4616,3084.488679,3095
4,코센,12725,1159,0,0,12725,1161,13942088000,-832452000,-5.634369,A009730,12725,1161.064077,1165


In [119]:
account.GetDataValue(0,0) ## 종목명
account.GetDataValue(1,0) ## 32 신용구분(현금결제는 32)
account.GetDataValue(2,0) ## ' ' 대출일
account.GetDataValue(3,0) ## 10 결제잔고
account.GetDataValue(4,0) ## 99300 결제장부단가
account.GetDataValue(5,0) ## 0 전일체결수량
account.GetDataValue(6,0) ## 0 금일체결수량
account.GetDataValue(7,0) ## 10 체결잔고수량
account.GetDataValue(8,0) ## 99536 장부가
account.GetDataValue(9,0) ## 992139000 평가금액 -> 계속해서 바뀌는 금액
account.GetDataValue(10,0) ## -3221000 평가손익 -> 계속해서 바뀌는 금액
account.GetDataValue(11,0) ## -0.3236702673059347 수익률 -> 계속해서 바뀌는 금액
account.GetDataValue(12,0) ## A000660 종목코드
account.GetDataValue(13,0) ## 32 주문구분
account.GetDataValue(14,0) ## 100022 
account.GetDataValue(15,0) ## 10 매도가능수량
account.GetDataValue(16,0) ## ' ' 만기
account.GetDataValue(17,0) ## 99536.124829 체결장부단가
account.GetDataValue(18,0) ## 100022 손익단가

100022

In [ ]:
account = win32com.client.Dispatch('CpTrade.CpTd6033')

acc

goods_type

account.SetInputValue(0, acc)
account.SetInputValue(1, goods_type)
account.SetInputValue(2, 50)
account.SetInputValue(3, '2')

account.BlockRequest()

account.GetDataValue(0,18)

unused_list = [1,2,13,14,16]
com_cnt = 0
while True:
    com_cnt += 1
    try:
        account.GetDataValue(0,com_cnt)
    except Exception as e:
        print('portfolio 내 종목개수 : {}개'.format(com_cnt))
        break

com_cnt

df = []
for i in range(com_cnt):
    row = []
    for j in range(19):        
        if j in unused_list:
            continue
        row.append(account.GetDataValue(j,i))
    df.append(row)


col = ['종목명', '결제잔고', '결제장부단가', '전일체결수량', ' 금일체결수량', ' 체결잔고수량', ' 장부가', ' 평가금액',
 '평가손익', '수익률', '종목코드', ' 매도가능수량', ' 체결장부단가', ' 손익단가']

df_account = pd.DataFrame(df, columns = col)

df_account.head()

In [1]:
from trader import trader

In [2]:
trading = trader()

Cybos가 정상적으로 연결되었습니다


In [3]:
trading.check_portfolio()

3700 모의투자 조회완료 되었습니다(cii.cif.astbln.selcnsbaseaccpt)
portfolio 내 종목개수 : 19개


,종목명,결제잔고,결제장부단가,전일체결수량,금일체결수량,체결잔고수량,장부가,평가금액,평가손익,수익률,종목코드,매도가능수량,체결장부단가,손익단가
0,SK하이닉스,10,99300,0,0,10,99536,992139000,-3221000,-0.323670,A000660,10,99536.124829,100022
1,삼성전자,10,61600,0,0,10,61773,602771000,-14959000,-2.421664,A005930,10,61773.054274,62095
2,이수화학,1597,9472,0,0,1597,9486,14634296000,-515202000,-3.400789,A005950,1597,9486.223844,9522
3,한창제지,4616,3080,0,0,4616,3084,12505692000,-1732306000,-12.166785,A009460,4616,3084.488679,3095
4,코센,12725,1159,0,0,12725,1161,14005466000,-769073000,-5.205397,A009730,12725,1161.064077,1165
5,성문전자,6551,2130,0,0,6551,2133,13539478000,-438299000,-3.135687,A014910,6551,2133.686148,2141
6,에이치엘비,138,102300,0,0,138,102449,14363948000,225965000,1.598286,A028300,138,102449.149358,102862
7,에코플라스틱,8513,1644,0,0,8513,1646,13354846000,-664863000,-4.742352,A038110,8513,1646.858975,1653
8,네오위즈홀딩스,1055,14294,0,0,1055,14315,14501413000,-601592000,-3.983261,A042420,1055,14315.645244,14370
9,코오롱생명과학,955,17950,0,0,955,17975,17122223000,-44761000,-0.260743,A102940,955,17975.901159,18046
